# Trop loud, pas faisable  


In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Recall, Precision
import tensorflow as tf
from tensorflow.image import flip_left_right, rot90

# === Config ===
# path_light = '/Volumes/DiskMorgane/light_data'
path_light ='/Volumes/DiskMorgane/bach_32/bach_1'
path_train_csv = '../data/csv_raw/train_labels.csv'

# === Cargar y procesar DataFrame ===
df = pd.read_csv(path_train_csv).copy()
df['motor_place'] = np.where(df['Motor_axis_0'] < 0, -1, (df['Motor_axis_0'] / df['Array_shape_axis_0'] * 32).astype(int))
df = df.groupby('tomo_id').first().reset_index()
#######################
# tomo_ids = ['tomo_00e047', 'tomo_0fe63f', 'tomo_1e9980']
# df = df[df['tomo_id'].isin(tomo_ids)]
# ##################
df

In [ ]:
expanded_rows = []
for _, row in df.iterrows():
    for i in range(32):
        new_row = row.copy()
        new_row['image_index'] = i
        new_row['motor_place'] = 1 if i == int(row['motor_place']) else 0
        expanded_rows.append(new_row)
df = pd.DataFrame(expanded_rows)
df

In [ ]:

# === Cargar imágenes y aplicar augmentación solo a positivos ===
def load_images(df, path_light_data, size=512, id_column='tomo_id'):
    images = []
    labels = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Loading images"):
        tomo_id = row[id_column]
        idx = row['image_index']
        label = row['motor_place']

        tomo_path = os.path.join(path_light_data, tomo_id)
        if not os.path.exists(tomo_path):
            continue

        image_files = sorted([
            f for f in os.listdir(tomo_path)
            if f.lower().endswith(('.png', '.jpg')) and not f.startswith("._")
        ])[:32]

        if idx >= len(image_files):
            continue

        img_path = os.path.join(tomo_path, image_files[idx])
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue

        img = cv2.resize(img, (size, size))
        img = np.expand_dims(img, axis=-1)
        images.append(img)
        labels.append(label)

        if label == 1:
            for _ in range(20):
                aug_img = img.copy()
                if np.random.rand() > 0.5:
                    aug_img = flip_left_right(aug_img).numpy()
                if np.random.rand() > 0.5:
                    aug_img = rot90(aug_img, k=np.random.randint(1, 4)).numpy()
                images.append(aug_img)
                labels.append(label)

    X = np.array(images, dtype=np.float32) / 255.0
    y = np.array(labels)
    return X, y

X, y = load_images(df, path_light_data=path_light, size=512)


Loading images:   5%|▌         | 1122/20736 [01:09<21:37, 15.12it/s]

In [5]:

# === División del dataset ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# === Modelo con DenseNet ===
def init_model_densenet(X):
    original_input_shape = X.shape[1:]
    base_input_shape = original_input_shape[:-1] + (3,) if original_input_shape[-1] == 1 else original_input_shape
    base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=base_input_shape)
    base_model.trainable = False

    inputs = Input(shape=original_input_shape)
    x = Lambda(lambda x: tf.image.grayscale_to_rgb(x))(inputs) if original_input_shape[-1] == 1 else inputs
    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=1e-4),
        metrics=['accuracy', Recall(), Precision()]
    )
    return model

# === Entrenamiento ===
def model_train(model, X_train, y_train, batch_size=16, epochs=5):
    es = EarlyStopping(patience=3, restore_best_weights=True)
    history = model.fit(
        X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_val, y_val),
        callbacks=[es],
        verbose=1
    )
    return history

# Inicializa y entrena
model = init_model_densenet(X)
history = model_train(model, X_train, y_train)